In [13]:
pip install --upgrade pip


   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/1.8 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/1.8 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/1.8 MB ? eta -:--:--
   ----------- -------------------

In [14]:
pip install selenium pandas openpyxl webdriver-manager


In [23]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time


In [24]:
def scrape_flipkart(search_query):
    products = []
    driver.get(f"https://www.flipkart.com/search?q={search_query.replace(' ', '+')}")
    
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div._13oc-S"))
        )
        time.sleep(2)
    except:
        print("⚠ Flipkart page load timeout")
        return products

    items = driver.find_elements(By.CSS_SELECTOR, "div._13oc-S")
    for item in items:
        try:
            name = item.find_element(By.CSS_SELECTOR, "div._4rR01T, a.s1Q9rs").text
            price = item.find_element(By.CSS_SELECTOR, "div._30jeq3").text
            try:
                rating = item.find_element(By.CSS_SELECTOR, "div._3LWZlK").text
            except:
                rating = "No Rating"
            products.append({
                "Website": "Flipkart",
                "Product Name": name,
                "Price": price,
                "Rating": rating
            })
        except:
            pass
    return products
    

In [25]:
def scrape_croma(search_query):
    products = []
    driver.get(f"https://www.croma.com/search/?text={search_query.replace(' ', '%20')}")
    
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "li.product-item"))
        )
        time.sleep(2)
    except:
        print("⚠ Croma page load timeout")
        return products

    items = driver.find_elements(By.CSS_SELECTOR, "li.product-item")
    for item in items:
        try:
            name = item.find_element(By.CSS_SELECTOR, "h3.product-title").text
            price = item.find_element(By.CSS_SELECTOR, "span.amount").text
            try:
                rating = item.find_element(By.CSS_SELECTOR, "span.rating").text
            except:
                rating = "No Rating"
            products.append({
                "Website": "Croma",
                "Product Name": name,
                "Price": price,
                "Rating": rating
            })
        except:
            pass
    return products

In [ ]:
if __name__ == "__main__":
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    
    search = input("Enter product to search: ")
    
    flipkart_data = scrape_flipkart(search)
    croma_data = scrape_croma(search)
    
    driver.quit()
    
    all_data = flipkart_data + croma_data
    if all_data:
        df = pd.DataFrame(all_data)
        filename = f"{search.replace(' ', '_')}_products.xlsx"
        df.to_excel(filename, index=False)
        print(f"✅ Data saved to {filename}")
        print(df.head())
    else:
        print("❌ No data found.")